In [178]:
%pip install --upgrade pip

%pip install networkx pandas openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [179]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import heapq
import numpy as np
import openpyxl 

In [180]:

df = pd.read_excel('./dataset/Data SC ACO.xlsx', sheet_name='Sheet1')

df['TV'] = df['Jarak (KM)'] / df['Kecepatan']

df.head()

,Kode Ruas,Node From,Node To,Kode Jalan,Nama Jalan,Jarak (KM),Vol max (q max),Vol (q),Kecepatan,TV
0,1,257,220,1,Raya Waru,2.101527,1011,962,56.5,0.037195
1,2,220,257,1,Raya Waru,2.101527,1014,962,56.5,0.037195
2,3,192,220,2,Letjend Sutoyo,0.385693,4382,4000,40.0,0.009642
3,4,220,192,2,Letjend Sutoyo,0.385693,4382,3422,40.0,0.009642
4,5,192,149,2,Letjend Sutoyo,0.795825,2124,1658,40.0,0.019896


In [181]:
depot_df = pd.read_csv('./dataset/depot.csv')

depot_df.head()

,Nama,Node,Persediaan,Time Begin,Time End
0,Depo 1,66,450,7:00,15:00
1,Depo 2,277,500,7:00,17:00
2,Depo 3,322,500,7:00,17:00


In [182]:
customer_df = pd.read_csv('./dataset/customer.csv')

customer_df.head()

,Nama,Node,Permintaan,Time Begin,Time End
0,Konsumen 1,1,25,7:00,8:00
1,Konsumen 2,7,30,7:00,8:00
2,Konsumen 3,12,20,7:00,8:00
3,Konsumen 4,21,35,9:00,9:00
4,Konsumen 5,46,30,8:00,10:00


In [183]:
def time_to_decimal(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours + minutes / 60

# Mengaplikasikan konversi waktu
depot_df['Time Begin'] = depot_df['Time Begin'].apply(time_to_decimal)
depot_df['Time End'] = depot_df['Time End'].apply(time_to_decimal)
customer_df['Time Begin'] = customer_df['Time Begin'].apply(time_to_decimal)
customer_df['Time End'] = customer_df['Time End'].apply(time_to_decimal)

In [184]:
depot_df

,Nama,Node,Persediaan,Time Begin,Time End
0,Depo 1,66,450,7.0,15.0
1,Depo 2,277,500,7.0,17.0
2,Depo 3,322,500,7.0,17.0


In [185]:
customer_df.head()

,Nama,Node,Permintaan,Time Begin,Time End
0,Konsumen 1,1,25,7.0,8.0
1,Konsumen 2,7,30,7.0,8.0
2,Konsumen 3,12,20,7.0,8.0
3,Konsumen 4,21,35,9.0,9.0
4,Konsumen 5,46,30,8.0,10.0


In [186]:
import networkx as nx

G = nx.Graph()

distance = {}  
tv_sum = {}  

for idx, row in df.iterrows():
    print(idx, row['Node From'], row['Node To'], row['Jarak (KM)'], row['TV'])
    G.add_edge(row['Node From'], row['Node To'], 
               distance=row['Jarak (KM)'], speed=row['Kecepatan'], TV=row['TV'])

print(f"Graph memiliki {G.number_of_nodes()} nodus dan {G.number_of_edges()} ruas jalan.")
print(G)

for idx, row in customer_df.iterrows():
    node = int(row['Node'])
    path_lengths, shortest_paths = nx.single_source_dijkstra(G, node, weight='distance')
    distance[node] = path_lengths
    
    tv_sum[node] = {}
    for dest, path in shortest_paths.items():
        total_tv = sum(G[u][v]['TV'] for u, v in zip(path[:-1], path[1:]))
        tv_sum[node][dest] = total_tv
    
    print(f"Distance for node {node}: {distance[node]}")
    print(f"Sum of TV for node {node}: {tv_sum[node]}")

for idx, row in depot_df.iterrows():
    node = int(row['Node'])
    path_lengths, shortest_paths = nx.single_source_dijkstra(G, node, weight='distance')
    distance[node] = path_lengths
    tv_sum[node] = {}
    for dest, path in shortest_paths.items():
        total_tv = sum(G[u][v]['TV'] for u, v in zip(path[:-1], path[1:]))
        tv_sum[node][dest] = total_tv
    
    print(f"Distance for node {node}: {distance[node]}")
    print(f"Sum of TV for node {node}: {tv_sum[node]}")

print("Final distance data:", distance)
print("Final TV sum data:", tv_sum)


0 257 220 2.1015272 0.03719517168141593
1 220 257 2.1015272 0.03719517168141593
2 192 220 0.3856929 0.0096423225
3 220 192 0.3856929 0.0096423225
4 192 149 0.7958252 0.01989563
5 149 192 0.7958252 0.01989563
6 149 124 0.4301281 0.0107532025
7 124 149 0.4301281 0.0107532025
8 220 222 1.1688458 0.02068753628318584
9 222 220 1.1688458 0.02068753628318584
10 222 229 0.4786694 0.00828147750865052
11 229 235 0.5578447 0.009651292387543253
12 235 377 0.3682297 0.006370756055363322
13 377 236 0.1056195 0.0018273269896193773
14 236 222 1.2992554 0.022478467128027684
15 377 378 0.6552817 0.011337053633217993
16 378 377 0.6552817 0.011337053633217993
17 378 247 0.244266 0.0042260553633218
18 247 251 0.0611755 0.0010583996539792388
19 251 378 0.3053636 0.0052831072664359865
20 251 245 0.3997919 0.006916814878892733
21 245 263 0.2864912 0.004956595155709343
22 263 258 0.3161118 0.0054690622837370246
23 258 251 0.3379633 0.005847115916955018
24 263 379 0.8223246 0.014227069204152249
25 379 263 0.822

In [187]:
import random

In [188]:
alpha = 0.7  # pheromone influence
beta = 0.7   # visibility influence
rho = 0.3  # pheromone evaporation rate
gamma = 0.7  # global pheromone decay rate
q0 = 0.95  # exploration vs exploitation threshold
num_ants = 300  # number of ants
max_iter = 100  # max iterations

# Node untuk Depo 1 dan konsumennya
depo1_start = 66
depo1 = [
    189, 174, 113, 46, 62, 60, 141, 81, 44, 39, 28, 140, 101, 4, 21, 254, 12, 7, 235, 144, 1, 104, 
]

depo2_start = 277
depo2 = [
    408, 95, 399, 296, 135, 292, 242, 205, 244, 161, 181, 175, 106,    
]

depo3_start = 322
depo3 = [
    210, 386, 248, 289, 298, 264, 221, 355, 256, 315, 402, 400, 333, 177, 376,     
]
inf = float('inf')
nodes = list(depo1)
N = len(nodes)

In [189]:



# nodes = 410

# distances = [[inf for _ in range(1, nodes)] for _ in range(1,nodes)]

# all_nodes = set(depo1.keys())  
# for connections in depo1.values():
#     all_nodes.update(connections)  

# node_to_index = {node: i for i, node in enumerate(all_nodes)}
# print(f"node_to_index {node_to_index}")

# distances = [[0 if i == j else get_distance_and_speed(i,j)[0] for j in range(1, nodes)] for i in range(1, nodes)]

# for node, connections in depo1.items():
#     current_index = node_to_index[node]
#     print(connections)
#     for connected_node in connections:
        
#         print(f"is {connected_node} in {node_to_index}?")
#         if connected_node in node_to_index:
#             connected_index = node_to_index[connected_node]
#             distances[current_index][connected_index],_ = get_distance_and_speed(current_index, connected_index) 
#             distances[connected_index][current_index],_ = get_distance_and_speed(connected_index, current_index) 
#             print(distances[current_index][connected_index])

# DEPO 1

In [190]:

LNN = 100  
tau0 = 1 / (N * LNN)  
pheromone = [[tau0 for _ in range(N)] for _ in range(N)]

def calculate_visibility(distances, depo1):
    N = len(depo1)
    visibility = []
    
    for i in range(N):
        row_visibility = []
        for j in range(N):
            if i == j:
                row_visibility.append(0)  # No visibility for the same node
            else:
                node_i = depo1[i]
                node_j = depo1[j]
                distance_ij = distances.get(node_i, {}).get(node_j, float('inf'))  # Get distance between nodes or inf if missing
                visibility_value = 1 / distance_ij if distance_ij > 0 else 0  # Inverse of distance as visibility
                row_visibility.append(visibility_value)
        visibility.append(row_visibility)
    
    return visibility

# Calculate visibility only for depo1
visibility_depo1 = calculate_visibility(distance, depo1)


def state_transition(pheromone, visibility_depo1, current_node, unvisited):
    q = random.random()
    unvisited_list = list(unvisited)
    if q <= q0:  # Eksploitasi
        next_node = max(unvisited_list, key=lambda j: (pheromone[current_node][j] ** alpha) * (visibility_depo1[current_node][j] ** beta))
    else:  # Eksplorasi
        probabilities = [(pheromone[current_node][j] ** alpha) * (visibility_depo1[current_node][j] ** beta) for j in unvisited]
        total_prob = sum(probabilities)

        if total_prob == 0:
            next_node = random.choice(unvisited_list)
        else:
            probabilities = [p / total_prob for p in probabilities]
            next_node = random.choices(unvisited_list, probabilities)[0]

    return next_node

def local_pheromone_update(pheromone, i, j):
    pheromone[i][j] = (1 - rho) * pheromone[i][j] + rho * tau0

def global_pheromone_update(pheromone, best_solution, best_cost, iteration_best_cost, third_best_cost):
    A = third_best_cost
    B = best_cost  
    C = iteration_best_cost 
    
    if A == 0:
        print("Warning: A (third_best_cost) is zero; skipping pheromone update.")
        return

    delta_tau = ((A - B) + (A - C)) / A

    for i, j in best_solution:
        pheromone[i][j] = (1 - gamma) * pheromone[i][j] + gamma * delta_tau

def acs():
    best_solution = None
    best_cost_distance = float('inf')
    best_cost_time = float('inf')
    third_best_cost_distance = float('inf')

    for iteration in range(max_iter):
        all_solutions = []
        all_costs_distance = []
        all_costs_time = []

        for ant in range(num_ants):
            current_node = random.randint(0, N-1)
            unvisited = set(range(N)) - {current_node}
            tour = [current_node]
            total_distance = 0
            total_time = 0

            while unvisited:
                next_node = state_transition(pheromone, visibility_depo1, current_node, unvisited)
                tour.append(next_node)
                

                distance_here = distance[66][depo1[current_node]]
                total_distance += distance_here
                total_time += tv_sum[66][depo1[current_node]]  # Calculate time using distance / speed

                local_pheromone_update(pheromone, current_node, next_node)
                current_node = next_node
                unvisited.remove(next_node)
                    

            all_solutions.append(tour)
            all_costs_distance.append(total_distance)
            all_costs_time.append(total_time)
        
        sorted_costs_distance = sorted(all_costs_distance)
        iteration_best_cost_distance = sorted_costs_distance[0]
        third_best_cost_distance = sorted_costs_distance[2] if len(sorted_costs_distance) >= 3 else third_best_cost_distance

        if iteration_best_cost_distance < best_cost_distance:
            best_cost_distance = iteration_best_cost_distance
            best_solution = all_solutions[all_costs_distance.index(best_cost_distance)]
            best_cost_time = all_costs_time[all_costs_distance.index(best_cost_distance)]

        global_pheromone_update(pheromone, zip(best_solution[:-1], best_solution[1:]), best_cost_distance, iteration_best_cost_distance, third_best_cost_distance)

        print(f"Iteration {iteration+1}: Best distance cost = {best_cost_distance} KM, Best time cost = {best_cost_time} hours")

    return best_solution, best_cost_distance, best_cost_time

# Jalankan ACS
best_solution, best_cost_distance, best_cost_time = acs()

best_solution_actual = [depo1[i] for i in best_solution]
print(f"Best solution found: {best_solution_actual} with distance cost {best_cost_distance} KM and time cost {best_cost_time} hours")


Iteration 1: Best distance cost = 78.9440479 KM, Best time cost = 1.6123941308959284 hours
Iteration 2: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 3: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 4: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 5: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 6: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 7: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 8: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 9: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 10: Best distance cost = 78.94404789999999 KM, Best time cost = 1.6123941308959282 hours
Iteration 11: Best distanc

# DEPO 2

In [191]:
nodes = list(depo2)
N = len(nodes)

alpha = 0.9  # pheromone influence
beta = 0.9   # visibility influence
rho = 0.1  # pheromone evaporation rate
gamma = 0.7  # global pheromone decay rate
q0 = 0.9  # exploration vs exploitation threshold
num_ants = 200  # number of ants
max_iter = 100  # max iterations

In [192]:

LNN = 100  
tau0 = 1 / (N * LNN)  
pheromone = [[tau0 for _ in range(N)] for _ in range(N)]

def calculate_visibility(distances, depo2):
    N = len(depo2)
    visibility = []
    
    for i in range(N):
        row_visibility = []
        for j in range(N):
            if i == j:
                row_visibility.append(0)  # No visibility for the same node
            else:
                node_i = depo2[i]
                node_j = depo2[j]
                distance_ij = distances.get(node_i, {}).get(node_j, float('inf'))  # Get distance between nodes or inf if missing
                visibility_value = 1 / distance_ij if distance_ij > 0 else 0  # Inverse of distance as visibility
                row_visibility.append(visibility_value)
        visibility.append(row_visibility)
    
    return visibility

# Calculate visibility only for depo2
visibility_depo2 = calculate_visibility(distance, depo2)


def state_transition(pheromone, visibility_depo2, current_node, unvisited):
    q = random.random()
    unvisited_list = list(unvisited)
    if q <= q0:  # Eksploitasi
        next_node = max(unvisited_list, key=lambda j: (pheromone[current_node][j] ** alpha) * (visibility_depo2[current_node][j] ** beta))
    else:  # Eksplorasi
        probabilities = [(pheromone[current_node][j] ** alpha) * (visibility_depo2[current_node][j] ** beta) for j in unvisited]
        total_prob = sum(probabilities)

        if total_prob == 0:
            next_node = random.choice(unvisited_list)
        else:
            probabilities = [p / total_prob for p in probabilities]
            next_node = random.choices(unvisited_list, probabilities)[0]

    return next_node

def local_pheromone_update(pheromone, i, j):
    pheromone[i][j] = (1 - rho) * pheromone[i][j] + rho * tau0

def global_pheromone_update(pheromone, best_solution, best_cost, iteration_best_cost, third_best_cost):
    A = third_best_cost
    B = best_cost  
    C = iteration_best_cost 
    
    if A == 0:
        print("Warning: A (third_best_cost) is zero; skipping pheromone update.")
        return

    delta_tau = ((A - B) + (A - C)) / A

    for i, j in best_solution:
        pheromone[i][j] = (1 - gamma) * pheromone[i][j] + gamma * delta_tau

def acs():
    best_solution = None
    best_cost_distance = float('inf')
    best_cost_time = float('inf')
    third_best_cost_distance = float('inf')

    for iteration in range(max_iter):
        all_solutions = []
        all_costs_distance = []
        all_costs_time = []

        for ant in range(num_ants):
            current_node = random.randint(0, N-1)
            unvisited = set(range(N)) - {current_node}
            tour = [current_node]
            total_distance = 0
            total_time = 0

            while unvisited:
                next_node = state_transition(pheromone, visibility_depo2, current_node, unvisited)
                tour.append(next_node)
                

                distance_here = distance[66][depo2[current_node]]
                total_distance += distance_here
                total_time += tv_sum[66][depo2[current_node]]  # Calculate time using distance / speed

                local_pheromone_update(pheromone, current_node, next_node)
                current_node = next_node
                unvisited.remove(next_node)
                    

            all_solutions.append(tour)
            all_costs_distance.append(total_distance)
            all_costs_time.append(total_time)
        
        sorted_costs_distance = sorted(all_costs_distance)
        iteration_best_cost_distance = sorted_costs_distance[0]
        third_best_cost_distance = sorted_costs_distance[2] if len(sorted_costs_distance) >= 3 else third_best_cost_distance

        if iteration_best_cost_distance < best_cost_distance:
            best_cost_distance = iteration_best_cost_distance
            best_solution = all_solutions[all_costs_distance.index(best_cost_distance)]
            best_cost_time = all_costs_time[all_costs_distance.index(best_cost_distance)]

        global_pheromone_update(pheromone, zip(best_solution[:-1], best_solution[1:]), best_cost_distance, iteration_best_cost_distance, third_best_cost_distance)

        print(f"Iteration {iteration+1}: Best distance cost = {best_cost_distance} KM, Best time cost = {best_cost_time} hours")

    return best_solution, best_cost_distance, best_cost_time

# Jalankan ACS
best_solution, best_cost_distance, best_cost_time = acs()

best_solution_actual = [depo2[i] for i in best_solution]
print(f"Best solution found: {best_solution_actual} with distance cost {best_cost_distance} KM and time cost {best_cost_time} hours")


Iteration 1: Best distance cost = 45.1087929 KM, Best time cost = 0.9201200821374265 hours
Iteration 2: Best distance cost = 45.1087929 KM, Best time cost = 0.9201200821374265 hours
Iteration 3: Best distance cost = 45.1087929 KM, Best time cost = 0.9201200821374265 hours
Iteration 4: Best distance cost = 45.10879289999999 KM, Best time cost = 0.9201200821374265 hours
Iteration 5: Best distance cost = 45.10879289999999 KM, Best time cost = 0.9201200821374265 hours
Iteration 6: Best distance cost = 45.10879289999999 KM, Best time cost = 0.9201200821374265 hours
Iteration 7: Best distance cost = 45.10879289999999 KM, Best time cost = 0.9201200821374265 hours
Iteration 8: Best distance cost = 45.10879289999999 KM, Best time cost = 0.9201200821374265 hours
Iteration 9: Best distance cost = 45.10879289999999 KM, Best time cost = 0.9201200821374265 hours
Iteration 10: Best distance cost = 45.10879289999999 KM, Best time cost = 0.9201200821374265 hours
Iteration 11: Best distance cost = 45.10

# DEPO 3

In [193]:
nodes = list(depo3)
N = len(nodes)

alpha = 0.7  # pheromone influence
beta = 0.9   # visibility influence
rho = 0.5  # pheromone evaporation rate
gamma = 0.7  # global pheromone decay rate
q0 = 0.9  # exploration vs exploitation threshold
num_ants = 200  # number of ants
max_iter = 100  # max iterations

In [195]:

LNN = 100  
tau0 = 1 / (N * LNN)  
pheromone = [[tau0 for _ in range(N)] for _ in range(N)]

def calculate_visibility(distances, depo3):
    N = len(depo3)
    visibility = []
    
    for i in range(N):
        row_visibility = []
        for j in range(N):
            if i == j:
                row_visibility.append(0)  # No visibility for the same node
            else:
                node_i = depo3[i]
                node_j = depo3[j]
                distance_ij = distances.get(node_i, {}).get(node_j, float('inf'))  # Get distance between nodes or inf if missing
                visibility_value = 1 / distance_ij if distance_ij > 0 else 0  # Inverse of distance as visibility
                row_visibility.append(visibility_value)
        visibility.append(row_visibility)
    
    return visibility

# Calculate visibility only for depo3
visibility_depo3 = calculate_visibility(distance, depo3)


def state_transition(pheromone, visibility_depo3, current_node, unvisited):
    q = random.random()
    unvisited_list = list(unvisited)
    if q <= q0:  # Eksploitasi
        next_node = max(unvisited_list, key=lambda j: (pheromone[current_node][j] ** alpha) * (visibility_depo3[current_node][j] ** beta))
    else:  # Eksplorasi
        probabilities = [(pheromone[current_node][j] ** alpha) * (visibility_depo3[current_node][j] ** beta) for j in unvisited]
        total_prob = sum(probabilities)

        if total_prob == 0:
            next_node = random.choice(unvisited_list)
        else:
            probabilities = [p / total_prob for p in probabilities]
            next_node = random.choices(unvisited_list, probabilities)[0]

    return next_node

def local_pheromone_update(pheromone, i, j):
    pheromone[i][j] = (1 - rho) * pheromone[i][j] + rho * tau0

def global_pheromone_update(pheromone, best_solution, best_cost, iteration_best_cost, third_best_cost):
    A = third_best_cost
    B = best_cost  
    C = iteration_best_cost 
    
    if A == 0:
        print("Warning: A (third_best_cost) is zero; skipping pheromone update.")
        return

    delta_tau = ((A - B) + (A - C)) / A

    for i, j in best_solution:
        pheromone[i][j] = (1 - gamma) * pheromone[i][j] + gamma * delta_tau

def acs():
    best_solution = None
    best_cost_distance = float('inf')
    best_cost_time = float('inf')
    third_best_cost_distance = float('inf')

    for iteration in range(max_iter):
        all_solutions = []
        all_costs_distance = []
        all_costs_time = []

        for ant in range(num_ants):
            current_node = random.randint(0, N-1)
            unvisited = set(range(N)) - {current_node}
            tour = [current_node]
            total_distance = 0
            total_time = 0

            while unvisited:
                next_node = state_transition(pheromone, visibility_depo3, current_node, unvisited)
                tour.append(next_node)
                

                distance_here = distance[66][depo3[current_node]]
                total_distance += distance_here
                total_time += tv_sum[66][depo3[current_node]]  # Calculate time using distance / speed

                local_pheromone_update(pheromone, current_node, next_node)
                current_node = next_node
                unvisited.remove(next_node)
                    

            all_solutions.append(tour)
            all_costs_distance.append(total_distance)
            all_costs_time.append(total_time)
        
        sorted_costs_distance = sorted(all_costs_distance)
        iteration_best_cost_distance = sorted_costs_distance[0]
        third_best_cost_distance = sorted_costs_distance[2] if len(sorted_costs_distance) >= 3 else third_best_cost_distance

        if iteration_best_cost_distance < best_cost_distance:
            best_cost_distance = iteration_best_cost_distance
            best_solution = all_solutions[all_costs_distance.index(best_cost_distance)]
            best_cost_time = all_costs_time[all_costs_distance.index(best_cost_distance)]

        global_pheromone_update(pheromone, zip(best_solution[:-1], best_solution[1:]), best_cost_distance, iteration_best_cost_distance, third_best_cost_distance)

        print(f"Iteration {iteration+1}: Best distance cost = {best_cost_distance} KM, Best time cost = {best_cost_time} hours")

    return best_solution, best_cost_distance, best_cost_time

# Jalankan ACS
best_solution, best_cost_distance, best_cost_time = acs()

best_solution_actual = [depo3[i] for i in best_solution]
print(f"Best solution found: {best_solution_actual} with distance cost {best_cost_distance} KM and time cost {best_cost_time} hours")

Iteration 1: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 2: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 3: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 4: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 5: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 6: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 7: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 8: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 9: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 10: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hours
Iteration 11: Best distance cost = 64.6730931 KM, Best time cost = 1.1986203719773256 hou